# Final RBFNN Model Script

This script runs the final RBFNN model, obtained by previous hyper-parameter search.

## 0) Importing Modules and MNIST Data:

In [1]:
# NumPy and SciPy modules:
import numpy as np
from scipy.ndimage import interpolation
# SciKit Learn Modules:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.linear_model  import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import PredefinedSplit
# Dask Modules (for Grid Search):
from dask.diagnostics import ProgressBar
from dask_searchcv import GridSearchCV
# Additional modules:
import random,time,gc

In [2]:
# Importing MNIST Data:
DATA_folder  = '../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

## 1) Definition of Scikit-Learn Transformer Objects:

### Definition of Deskewing Transformer
Transformer that applies a deskewing (2D shear + recentering) transformation on the images.

In [3]:
# SKLEARN Compatible Transformer - Image Deskewing Transformation
class myDeskewer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization
    def __init__(self, debug=False):
        self.debug = debug
    
    # No fitting required:
    def fit(self, X=None, y=None):        
        return self
    
    # Definition of Moments Calculation Function
    @staticmethod
    def moments(image):
        # Create mesh grids for centroid calculation:
        c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
        # Calculate Total Image weight:
        totalImage = np.sum(image) #sum of pixels
        # Calculate image centroids:
        m0 = np.sum(c0*image)/totalImage # mu_x
        m1 = np.sum(c1*image)/totalImage # mu_y
        # Calculate image variance and covariance
        var = np.sum((c0-m0)**2*image)/totalImage      # Var(x)
        cov = np.sum((c0-m0)*(c1-m1)*image)/totalImage # Cov(x,y)
        mu_vector = np.array([m0,m1])
        alpha = cov/var
        return mu_vector, alpha

    # Definition of Deskewing Function:
    @staticmethod
    def deskew(img):
        image = img.reshape(28,28)
        c,alpha = myDeskewer.moments(image)
        # Define transformation matrix:
        affine = np.array([[1,0],[alpha,1]])
        # Define transformation offset:
        ocenter = np.array(image.shape)/2.0
        offset = c-np.dot(affine,ocenter)
        # Return affine_transformation
        return interpolation.affine_transform(image,affine,offset=offset).reshape(28*28)
    
    # Clculate Kernel Components based on obtained model
    def transform(self, X, y=None):
        
        if self.debug:
            print('Applying Deskewing Transformation!!!')
            t = time.time()
        # Preallocate memory:
        new_data = np.empty(X.shape)
        
        # Loop over each image:
        for k in range(X.shape[0]):
            new_data[k]=myDeskewer.deskew(X[k])
            
        if self.debug:
            print('Deskewing Transformation finished in: ',(time.time()-t))
        
        return new_data
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

### Definition of Mini-Batch Kernel PCA Transformer
Transformer that applies Kernel PCA transformation on a randomly sampled data mini-batch (due to high memory requirements).

In [4]:
# Due to high memory requirements - we cannot process the whole dataset with kPCA - we need to sample a random batch!!!

# SKLEARN Compatible Transformer - supports fit method (finding kPCA) and transform method (applying kPCA)
class myKernelPCA(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 250 dimensions)
    def __init__(self, n_kPCA=40, kernel = 'laplacian', degree=3, whiten=True, debug=False):
        self.n_kPCA    = n_kPCA # number of kernel PCA dimensions to be retained
        self.kernel    = kernel # kernel type (rbf, poly, cosine)
        self.degree    = degree # poly kernel degree
        self.whiten    = whiten and (kernel!='laplacian') and (kernel!='rbf')
        self.debug     = debug  # debug flag
        self.max_smpls = 250    # max number of samples per digit
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.kPCA = KernelPCA(n_components=self.n_kPCA,kernel=self.kernel,degree=self.degree,copy_X=False,remove_zero_eig=True,random_state=1389)
        
        if self.debug:
            print('Fitting Kernel PCA !!!')
            t = time.time()
        
        # If we have less than max number of smpls - fit regular kPCA
        if X.shape[0]<(self.max_smpls*10):
            self.kPCA.fit(X)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(X))
        
        # Else randomly pick max number of smpls for each digit and fit
        else:
            # Fix the random seed (for repeatable results)
            prng = np.random.RandomState()
            prng.seed(654)
            # Check how many smps are available per digit
            l = []
            for dig in range(10):
                l.append(min(self.max_smpls,sum(y==dig)))
            # Preallocate memory
            new_data = np.empty((sum(l),X.shape[1]))
            # Fill with randomly drawn smpls
            st = 0
            for dig in range(10):
                en = st+l[dig]
                data  = X[y==dig]
                smpls = prng.choice(data.shape[0],l[dig],replace=False)
                new_data[st:en,] = data[smpls,]
                st = en
            # Fit KernelPCA to new_data
            self.kPCA.fit(new_data)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(new_data))
            # Release old vars, and collect garbage
            del data, smpls, new_data
            gc.collect()
            
        if self.debug:
            print('Kernel PCA fitted in: ',(time.time()-t))
        
        return self
    
    # Clculate Kernel Components based on obtained model
    def transform(self, X, y=None):
        
        if self.debug:
            print('Applying Kernel PCA transformation!!! (data', X.shape,')')
            t = time.time()
        # Preallocate memory:
        new_data = np.empty((X.shape[0],self.kPCA.lambdas_.shape[0]))
        st = 0
        ch = 200
        ns = X.shape[0]
        # Transform data in chunks of 200 smpls (memory hungry transformation)
        while st<ns:
            en = min(st+ch,ns)
            new_data[st:en,:] = self.kPCA.transform(X[st:en,:])
            st = en
        
        if self.whiten:
            new_data = self.scaler.transform(new_data)
                
        if self.debug:
            print('Kernel PCA transformed in: ',time.time()-t)
        
        return new_data
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

### Definition of RBF Transformer (sklearn compatible object):
Transformer that applies RBF transformation, by first finding the required RBF centers, and then calculating the distance-based activation.

In [5]:
# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_centers=250, rn_ratio=0.1, cl_ratio=1.0, debug=False):
        self.n_centers = n_centers              # number of data centers to be formed per digit
        self.rn_ratio  = min(1,max(0,rn_ratio)) # random  ratio (between Random Selection and Clustering)
        self.cl_ratio  = min(1,max(0,cl_ratio)) # cluster ratio (between kMeans and Agglomerative)
        self.debug     = debug                  # debug flag
        if self.debug:
            print(self.n_centers,self.cl_ratio)
    
    # Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
    @staticmethod
    def form_clusters(data, n_kmeans, n_agglo, n_random, dig):

        n_km = min(n_kmeans,data.shape[0])
        n_ag = min(n_agglo ,data.shape[0])
        n_rn = min(n_random,data.shape[0])

        lbls_set = []

        if n_km>0:
            kmeans = KMeans(n_clusters=n_km, random_state=0, init='k-means++', algorithm='elkan')
            lbls_kmeans = kmeans.fit_predict(data)
            lbls_set.append(lbls_kmeans)

        if n_ag>0:
            #agglo  = SpectralClustering(n_clusters=n_ag, affinity='nearest_neighbors',eigen_solver='arpack',random_state=456,assign_labels='discretize')
            agglo  = SpectralClustering(n_clusters=n_ag, affinity='sigmoid',eigen_solver='arpack',random_state=456,assign_labels='discretize')
            lbls_agglo  = agglo.fit_predict(data)
            lbls_set.append(lbls_agglo)

        centers     = []

        # Find cluster centers and covar matrix:
        for lbls in lbls_set:
            for k in range(max(lbls)+1):
                cluster = data[lbls==k,:]
                centers.append(cluster.mean(axis=0))

            del cluster

        # Add random points as centers:
        if n_rn>0:
            # seed a new random generator (to get repeatable results for hyperparam tuning)
            prng = np.random.RandomState()
            prng.seed(dig*n_rn)
            smpls = prng.choice(data.shape[0],n_rn,replace=False)
            for s in smpls:
                centers.append(data[s,:])

        return centers;
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.centers     = [] # list of cluster center vectors
        self.num_centers = [] # list of number of centers per digit
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_random   = round(self.rn_ratio*self.n_centers)
        n_clusters = self.n_centers - n_random
        n_kmeans   = round(self.cl_ratio*n_clusters)
        n_agglo    = n_clusters - n_kmeans
        
        if self.debug:
            print('Clustering data ',(n_kmeans,n_agglo))
            t = time.time()
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers = myRBFtransformer.form_clusters(data, n_kmeans, n_agglo, n_random, dig)
            self.centers.extend(centers)
            self.num_centers.append(len(centers))
            
        if self.debug:
            print('Clustering time: ',(time.time()-t))
        
        del data, centers
        gc.collect()
        
        return self
    
    # Function to compute whole Fi output for given dataset (for all RBF centers)
    @staticmethod
    def fi_transform(data, all_centers):
        # data        - given dataset matrix for which to compute fi values
        # center      - list of all center vectors on which to compute fi vals

        new_data = np.empty((data.shape[0],len(all_centers)))
        st = 0
        ch = 200
        ns = data.shape[0]
        # Process data in chunks of 200 smpls (optimal speed)
        while st<ns:
            en = min(st+ch,ns)
            for k in range(len(all_centers)):
                # # SAME DEVIATION for ALL DIMENSIONS (better much better than cluster separate):
                new_data[st:en,k] = (np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],en-st,axis=0))).mean(axis=1)
            new_data[st:en,:] = np.exp(-np.sqrt(new_data[st:en,:]))
            st = en
            gc.collect()
        return new_data
    
    # Computes fi values (with Gaussian function) based on obtained data centers
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        if self.debug:
            print('Calculating all Fi outputs !!! (data', X.shape,')')
            t = time.time()
        all_fis = myRBFtransformer.fi_transform(X, self.centers)
        
        if self.debug:
            print('Fi calculation time: ',time.time()-t)
        
        return all_fis
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

## 3) Layer Linking (pipeline):
Merging all layers in a single model.

In [6]:
n_kPCA    = 40
kernel    = 'laplacian'
n_centers = 50
# n_centers = 250
rn_ratio  = 0.5
cl_ratio  = 1.0
epochs    = 75
C         = 1e6
tol       = 1e-6
debug     = False

dskw    = myDeskewer(debug=debug)
scaler  = StandardScaler()
pca     = myKernelPCA(kernel=kernel,n_kPCA=n_kPCA,debug=debug)
rbf     = myRBFtransformer(n_centers=n_centers,rn_ratio=rn_ratio,cl_ratio=cl_ratio, debug=debug)
# scaler2 = StandardScaler()
# clsf    = MLPClassifier(hidden_layer_sizes=(1000,), random_state=1, batch_size=512 , max_iter=epochs, tol=-1)
# clsf    = LogisticRegression(tol=tol,C=C,random_state=12,solver='newton-cg',multi_class='multinomial',max_iter=100)
clsf    = LogisticRegression(tol=tol,C=C,random_state=12,solver='liblinear')

In [7]:
# pipe = Pipeline(steps=[('dskw',dskw),('scal', scaler),('pca', pca),('rbf',rbf),('scal2', scaler2),('clsf',clsf)])
pipe = Pipeline(steps=[('scal', scaler),('rbf',rbf),('clsf',clsf)])

## 4) Model Fitting:

In [8]:
# Fitting the model to training data:

t = time.time()
pipe.fit(data,lbls)
print(time.time()-t)

2706.731872320175


In [9]:
# Predicting the test data:

pred_test_lbls = pipe.predict(test_data)

In [10]:
# Printing the Classification Report:

target_names = ['digit 0','digit 1','digit 2','digit 3','digit 4','digit 5','digit 6','digit 7','digit 8','digit 9']
print(classification_report(test_lbls, pred_test_lbls, target_names=target_names))

             precision    recall  f1-score   support

    digit 0       0.97      0.98      0.98       980
    digit 1       0.99      0.99      0.99      1135
    digit 2       0.94      0.94      0.94      1032
    digit 3       0.95      0.94      0.94      1010
    digit 4       0.95      0.95      0.95       982
    digit 5       0.94      0.93      0.93       892
    digit 6       0.96      0.96      0.96       958
    digit 7       0.93      0.95      0.94      1028
    digit 8       0.92      0.94      0.93       974
    digit 9       0.94      0.92      0.93      1009

avg / total       0.95      0.95      0.95     10000



In [11]:
# Print the final score:

print(pipe.score(test_data,test_lbls)*100)

94.98
